In [ ]:
x = df_subset.drop(['M_WEATHER', 'TIMESTAMP'], axis=1).values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)

df_norm = pd.DataFrame(x_scaled, columns=df_subset.drop(['M_WEATHER', 'TIMESTAMP'], axis=1).columns.tolist())
df_norm['TIMESTAMP'] = pd.to_datetime(df_subset['TIMESTAMP'], unit='s', dayfirst=True).reset_index(drop=True)
df_norm['M_WEATHER'] = df_subset['M_WEATHER'].reset_index(drop=True)
df_norm = df_norm.set_index('TIMESTAMP')
df_norm

In [ ]:
df_norm['M_WEATHER'] = df_norm['M_WEATHER'].apply(lambda x: str(x))
df_norm

In [ ]:
df_norm.dropna(inplace=True)

In [ ]:
df = df_norm
Y = df['M_WEATHER']
X = df.drop(columns=['M_WEATHER'],inplace=False)

In [ ]:
X, X_test, y, y_test = train_test_split(X,Y,test_size = 0.2,train_size = 0.8,random_state = 42)
X_train, X_cv, y_train, y_cv = train_test_split(X,y,test_size = 0.25,train_size = 0.75,random_state = 42)

In [ ]:
model = RandomForestClassifier(class_weight = 'balanced')
model.fit(X_train,y_train)

In [ ]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X_cv, y_cv, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [ ]:
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test,predictions)
print(accuracy)

In [ ]:
row = X.iloc[1].to_numpy().reshape(1,-1)
yhat = model.predict(row)
print('predicted weather is',yhat)

In [ ]:
filename = "finalized_randomforest_classifier_model.sav"
pickle.dump(model, (open(filename, "wb")))